In [ ]:
# Source: https://github.com/bnsreenu/python_for_microscopists/blob/master/263_Object%20localization%20in%20images%E2%80%8B_using_GAP_layer/263_Object%20localization%20in%20images%E2%80%8B_using_GAP_layer.py
import numpy as np
import scipy   #to upscale the image
import matplotlib.pyplot as plt
import cv2     
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.models import Model   
from PIL import Image

In [1]:
#Import the resnet50 model
model = ResNet50(weights='imagenet')
print(model.summary()) #Notice the Global Average Pooling layer at the last but one

NameError: name 'ResNet50' is not defined

In [ ]:
# Get weights for the prediction layer (last layer)
# We should see 2048 weights for each of the 1000 classes (2048,1000)
last_layer_weights = model.layers[-1].get_weights()[0]  #Predictions layer

#Output both predictions (last layer) and conv5_block3_add (just before final activation layer)
ResNet_model = Model(inputs=model.input, outputs=(model.layers[-4].output, model.layers[-1].output)) 

In [ ]:
def get_heatmap(input_img):
    img_tensor = np.expand_dims(input_img, axis=0)
    preprocessed_img = preprocess_input(img_tensor)
    
    #Get the predictions and the output of last conv. layer. 
    last_conv_output, pred_vec = ResNet_model.predict(preprocessed_img)
    #Last conv. output for the image
    last_conv_output = np.squeeze(last_conv_output) #7x7x2048
    #Prediction for the image
    pred = np.argmax(pred_vec)
    
    decode_predictions(pred_vec, top=1)
    
    # spline interpolation to resize each filtered image to size of original image 
    h = int(img.shape[0]/last_conv_output.shape[0])
    w = int(img.shape[1]/last_conv_output.shape[1])
    upsampled_last_conv_output = scipy.ndimage.zoom(last_conv_output, (h, w, 1), order=1) # dim: 224 x 224 x 2048
    
    #Get the weights from the last layer for the prediction class
    last_layer_weights_for_pred = last_layer_weights[:, pred] # dim: (2048,) 
    
    #To generate the final heat map. 
    #Reshape the upsampled last conv. output to n x filters and multiply (dot product) 
    # with the last layer weigths for the prediction. 
    # Reshape back to the image size for easy overlay onto the original image. 
    heat_map = np.dot(upsampled_last_conv_output.reshape((224*224, 2048)), 
                  last_layer_weights_for_pred).reshape(224,224) # dim: 224 x 224
    return heat_map

In [ ]:
video_dir = '../../video/1/'
fname = '2022-04-19 15-44-38.mp4'

sample = cv2.VideoCapture(video_dir+fname)

In [ ]:
skip_frames = 20
i = 0

def play_video_format(video_data):
    # Read until video is completed
    while(video_data.isOpened()):
        # Capture frame-by-frame
        ret, frame = video_data.read()
        
        # skip frames
        if (i < skip_frames):
            i += 1
            continue
        i = 0
        img = Image.fromarray(img, 'RGB')
        img = img.resize((224, 224))
        img = np.array(img)
        
        if ret == True:
            # Display the resulting frame
            heap_map = get_heatmap(img)
            fig, ax = plt.subplots()
            ax.imshow(img)
            ax.imshow(heat_map, cmap='jet', alpha=0.5)
            plt.show()
        # Break the loop
        else:
            break

In [ ]:
play_video_format(sample)